<div style="width:100%; background-color: #000041"><a target="_blank" href="http://university.yugabyte.com"><img src="assets/YBU_Logo.png" /></a></div>

> **YugabyteDB YSQL Development**
>
> Enroll for free at [Yugabyte University](https://university.yugabyte.com/).
>

# Demystifying table sharding, tablets, and data distribution
As a distributed SQL database, YugabyteDB stores data differently than a stand-alone, monolithic  database. Because data is stored differently, YugabyteDB reads and writes data differently as well.

Like a monolithic relational databases, YugabyteDB stores data in tables. However, in YugabyteDB, a distributed database, a table consists of tablets. 

A tablet represents a table shard which contains a set of rows for the logical table. Tablets for a table often exists in a peer group of leader and followers. This group of tablet peers exists as a Raft consensus group. Each tablet is a customized RocksDB instance. RocksDB is a persistent key-value database. This architecture of a table existing as tablet peers in a Raft consensus group where each tablet is a customized RocksDB key-value database is called DocDB, Yugabyte's distributed document store.

In this notebook, using Explain Plans, built-in functions, and custom utilities for YB-TServer metrics, you will learn:
- how YugabyteDB stores data for a table in one or more tablets using either hash consistent sharding or range sharding
- how YugabyteDB reads tablet data during query execution
- how data persist to disk as SST files
- the internals of SST files as both DocKey and SubDocKey

## 🛠️ Requirements
Here are the requirements for this notebook:
- ✅ Create the notebook variables in `01_Lab_Setup.ipynb`, which you previously did
- ✅ Create the `ds_ybu` database, which you previously did
- ☑️ Import the notebook variables, *which you must do next*
- ☑️ Connect to the `ds_ybu` database, *which you must do next*
- ☑️ Complete the following sections
  -  Hash Sharding
  -  Range Sharding
  -  Comparing hash and range sharding
  -  Summary



### Select your notebook kernel
- In the Notebook toolbar, click **Select Kernel**.
<br>
<img width=50% src="assets/01_01_Select_Kernel_Toolbar.png" />

- Next, in the dropdown, select **Python 3.8.13** or higher.
<br>
<img width=50% src="assets/01_02_Select_Kernel_Dropdown.png" />

That's it!

## ⛑️ Getting help
The best way to get help from the Yugabyte University team is to post your question on YugabyteDB Community Slack in the #training or #yb-university channels. To sign up, visit [https://communityinviter.com/apps/yugabyte-db/register](https://communityinviter.com/apps/yugabyte-db/register).

---
## 👣 Setup steps
Here are the steps to setup this lab:
- Import the notebook variables
- Connect to `db_ybu` database
- Load the SQL Magic extension for the connection
- Create the prepared statements

### Import the notebook variables 

> ‼️ Important
> 
> Do **NOT** skip running the following cell. 
> 

The following Python cell reads the stored variables created in the `01_Lab_Setup.ipynb` notebook. To run the script, select Execute Cell (Play Arrow) in the left gutter of the cell.

In [ ]:
# Use %store -r to read 01_Lab_Requirements_Setup variables
%store -r MY_DB_NAME
%store -r MY_YB_PATH
%store -r MY_YB_PATH_DATA
%store -r MY_GITPOD_WORKSPACE_URL
%store -r MY_HOST_IPv4_01
%store -r MY_HOST_IPv4_02
%store -r MY_HOST_IPv4_03
%store -r MY_NOTEBOOK_DIR
%store -r MY_TSERVER_WEBSERVER_PORT
%store -r MY_NOTEBOOK_DATA_FOLDER
%store -r MY_YB_MASTER_HOST_GITPOD_URL
%store -r MY_YB_TSERVER_HOST_GITPOD_URL

## Connect to the `db_ybu` database
Run all the cells in this section:
- Connect using Python and PostgreSQL driver
- Load the SQL magic extension
- Create the prepared statements

### Connect using Python and PostgreSQL driver

In [ ]:
# connect use Python 3.7.9+
import psycopg2
import sqlalchemy as alc
from sqlalchemy import create_engine

db_host=MY_HOST_IPv4_01
db_name=MY_DB_NAME

connection_str='postgresql+psycopg2://yugabyte@'+db_host+':5433/'+db_name

# engine = create_engine(connection_str)

### Load the SQL magic extension

In [ ]:
%reload_ext sql

# SQL magic for python connection string
%sql {connection_str}

### Create the prepared statements

> IMPORTANT!
>   
> In order to create the prepared statements for the SQL magic connection, you must run the following cell!!!
> 
> Do not skip this step.
> 

In [ ]:
#%% python, but prepared statements as sql magic

if (MY_YB_MASTER_HOST_GITPOD_URL is None):
    a = %sql select fn_yb_create_stmts()
else:
    a = %sql select fn_yb_create_stmts(:MY_YB_MASTER_HOST_GITPOD_URL )
print (a)

Confirm that the following query returns a count of 3 (for three prepared statements).

In [ ]:
%%sql 
select count(*) from pg_prepared_statements where 1=1 and name in ('stmt_util_metrics_snap_tablet','stmt_util_metrics_snap_table','stmt_util_metrics_snap_reset')

---
## Hash Sharding

### q1 | Create a table with no Primary Key (PK)
For this first example, without explicitly defining a primary key, you will create a table. Then, you will review the Explain Plan for a query, and also review the RocksDB metrics for the query.

In [ ]:
%%sql /* create table no PK and insert rows */
drop table if exists tbl_no_pk;

create table if not exists tbl_no_pk (k int, v text);

insert into tbl_no_pk (k,v)
select g.id, format('%s%s',chr(97+CAST(random() * 25 AS INTEGER)),chr(97+CAST(random() * 25 AS INTEGER)))
from generate_series(1, 1000) AS g (id);

Describe the table. Where is the Primary Key?

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" #\d+
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

./bin/ysqlsh -d ${DB_NAME} -c "\d+ tbl_no_pk"

#### q1a | Query the table for a value of k

The following is a query that will show the Explain Plan for the query. The first statement resets the metrics that will be captured after running the query. The query itself has a `where` clause predicate for `k` . 

In [ ]:
%%sql /* explain plan */
-- beta feature enabled by default, so turning off
SET yb_enable_expression_pushdown=off;
SHOW yb_enable_expression_pushdown;


execute stmt_util_metrics_snap_reset;
explain (analyze, costs off, verbose, timing on) 
select '' _
    , k
    , v
from tbl_no_pk 
where 1=1 
and  k=1000
-- limit 30
;

##### q1a | Explain Plan (above ^^)
Because YugabyteDB reuses the PostgreSQL query layer, it is possible to view an Explain Plan for a query. The query optimizer generates the Explain Plan for the stated query execution.
The Explain Plan for this query shows:
- `Seq Scan on public.tbl_no_pk (actual time=7.912..11.720 rows=1 loops=1)`

A `Seq Scan` is a full table scan, meaning that potentially all the rows in the table will be read.

- `Rows Removed by Filter: 999`

999 rows were removed, leaving the final single row.

The Explain Plan infers how the query accesses tablet data, but does **NOT** show the metrics for tablet access. Using a custom utility, you can view the metrics for the query execution in the notebook cell below. To view the metrics, run the following cell that contains the prepared statement:

In [ ]:
%%sql /* metrics */
execute stmt_util_metrics_snap_table;

##### q1a | Metrics (above ^^)

The Explain Plan revealed that in order to return the single row result, the query first reads 1000 rows and then filters out 999 rows.

The Metrics reports shows that the query accesses 3 tablets for the table, `tbl_no_pk`. Each tablet has 1 seek of an offset in RocksDB, and then per offset, 900+ reads.

Here is how to read the output of the Metrics report:
- row_name `db_ybu | tbl_no_pk | https://7000-yugabytedbu-ysqlpe03whe-um13wtg7p0o.ws-us54.gitpod.io/table?id=00004000000030008000000000004039 | bc64657cf03842fab00a416cc67c8413 | L`
  - database --> `db_ybu`
  - table or index or materialized view  --> `tbl_no_pk`
  - table_id --> `https://7000-yugabytedbu-ysqlpe03whe-um13wtg7p0o.ws-us54.gitpod.io/table?id=00004000000030008000000000004039` --> url to browser -->  table_id? --> `00004000000030008000000000004039`
  - tablet_id --> `4aed1608c05d40edb7105b11abc629cd`
  - isLeader --> `L`
- rocksdb_number_db_seek
  - The number of Seek() RocksDB API calls, which means the number of seeks for offsets 
- rocksdb_number_db_next
  - The number of Next() RocksDB API calls, which means the number of reads from the offset
- rows_inserted
  - number of rows inserted 

### View the Table details in the YB-Master web ui
To better understand the Metrics report, you can view the details of a table in the YB-Master web ui.

> Note
>  
>  In gitpod, only the `127.0.0.1` host is available on port 7000, and will show as `https://7000-yugabytedbu-ysqlpe03whe-um13wtg7p0o.ws-us54.gitpod.io/table?id=00004000000030008000000000004039` . You are not able to browse `127.0.0.2` or `127.0.0.3`

Run the cell  below to get the URL to open in your web browser. If you are running cell this locally, comment and uncomment as directed.

In [ ]:
#%% python, but prepared statements as sql magic
THIS_TABLE_NAME = 'tbl_no_pk'
THIS_SCHEMA_NAME = 'public'
DB_NAME = MY_DB_NAME

## Comment out if local
view_gitpod_url = %sql select fn_get_table_id_url(:MY_YB_MASTER_HOST_GITPOD_URL,7000,:DB_NAME,:THIS_SCHEMA_NAME,:THIS_TABLE_NAME ) as view_gitpod_url
print (view_gitpod_url)

## Uncomment if local
# view_local_url = %sql select fn_get_table_id_url(:MY_HOST_IPv4_01,7000,:DB_NAME,:THIS_SCHEMA_NAME,:THIS_TABLE_NAME ) as view_local_url
# print (view_local_url)

#### Review the Column section
The column section shows details about each column in the table, index, or materialized view. In the column section for `tbl_no_pk`, you will see:


| Column | ID	| Type |
|--------|------|------|
| ybrowid    | 0	| binary NOT NULL PARTITION KEY | 
| k	         | 1	| int32 NULLABLE NOT A PARTITION KEY | 
| v	         | 2	| string NULLABLE NOT A PARTITION KEY | 

<br/>

> Important!
>  
> When you create a table without a Primary Key (PK), YugabyteDB creates a hidden primary key for you, `ybrowid`. You will see this hidden PK in the YB-Master web ui for the `tbl_no_pk` table.


`ybrowid` functions as a hidden PK for a row with a random Universal Unique Identifier (UUID) value. Using  `\d` or `\d+` will not show the `ybrowid` primary key. It is not possible to query the `ybrowid` value.

In the case of the `tbl_no_pk` table,  `ybrowid` is a hidden primary key that functions as the partition key.

##### Partition Key
When YugabyteDB processes an insert query for a row of data into a table with a consistent hash sharding strategy, YugabyteDB uses the shard key (shown as `PARTITION KEY`) to distribute the data among the tablet leaders. 

With consistent hash sharding, a partitioning algorithm distributes data evenly and randomly across shards. By computing a consistent hash on the partition key (or keys) of a given row, YugabyteDB knows where to insert the row among the tablet leaders.



#### Review the Tablet section
For the given table, the Tablet section shows the details for the existing tablets. Of particular interest are the number of tablet leaders and the partition strategy. Here is an abridged example of the Tablet section for the `tbl_no_pk` table:

| Tablet ID |	Partition	| SplitDepth	| State	| Hidden	| Message	| RaftConfig|
|--|--|--|--|--|--|--|
| some_uuid_1 |	`hash_split: [0x5555, 0xAAAA)` |	0	| Running|	false| Tablet reported with an active leader	|<li>FOLLOWER: 127.0.0.1 <li>FOLLOWER: 127.0.0.3<li>LEADER: 127.0.0.2  |
| some_uuid_2	| `hash_split: [0xAAAA, 0xFFFF)`	| 0 |  Running |false |	Tablet reported with an active leader |<li>FOLLOWER: 127.0.0.1 <li>LEADER: 127.0.0.3 <li>FOLLOWER: 127.0.0.2 |
| some_uuid_3 <br>(tablet leader where the row lives) |	`hash_split: [0x0000, 0x5555)` |	0 |	Running | 	false	| Tablet reported with an active leader |	<li>LEADER: 127.0.0.1<li>FOLLOWER: 127.0.0.3<li>FOLLOWER: 127.0.0.2 |

##### Why are there three tablets each with a tablet_id ?

Various properties and configurations determine the number of tablets for a specific table in a given YugabyteDB cluster. 

Your YugabyteDB cluster is a three node cluster. Each node runs two processes: a YB-Master process and a YB-TServer process. Your cluster has a replication factor of three, which means that each tablet leader will have two tablet followers. 

Certain flags specify the number of shards per YB-TServer. Your cluster has two specific flags: 
-  `yb_num_shards_per_tserver=1`
-  `ysql_num_shards_per_tserver=1`

These flags dictate that each YB-TServer process will initially create just one tablet leader for a given table with hash sharding.

For this reason, with your three node cluster, a table will have a single tablet leader per node.  And since there are three nodes in your YugabyteDB cluster, there will be a total of three tablet leaders for a given table with hash sharding. The replication factor of three specifies that each tablet leader will have two tablet followers. 

##### What is the `hash_split` value in the Partition column? 

For a table with hash sharding, each tablet has a Partition value that shows a  `hash_split` . The `hash_split` details a range of hexadecimal values. 
```
hash_split: [0x0000, 0x5555)
hash_split: [0x5555, 0xAAAA)
hash_split: [0xAAAA, 0xFFFF)
```
The conversion of the above hexadecimal values to integer values is as follows:
```
0-21845
21845-43690
43690-65535
```

In other words, the Partition value for hash sharding represents a range of hexadecimal values starting with 0 and ending with 65535. Here is illustration of consistent hash sharding:

<img  style="background-color:#FFF" src="assets/hash_sharding.png" >

In this illustration, the primary key (PK) is the partition key. In the table above, there is no declared primary key, but there is `ybrowid`. `ybrowid` functions as a hidden PK for a row with a random Universal Unique Identifier (UUID) value. As a result,  `ybrowid` serves as the partition key for the table without a declared PK.

The hash value of the partition key results in a hexadecimal value. This value falls between the lower and upper values of a `hash_split` range for a tablet leader. 

YSQL has an built-in function that shows the integer form of the hash code: `yb_hash_code()` . This built-in function returns an integer, making it easier to interpret the hexadecimal values of a hash_splt range.

However, because YugabyteDB does not expose `ybrowid` to a database user, it is not possible to use `yb_hash_code()` to determine what is the integer value for the hash code of `ybrowid`. 

### q2 | Alter the table and add a Primary Key
To help illuminate how YugabyteDB distributes data, you can a create primary key  for the `k` column of the table. Later, you will use the `yb_code_hash()` built-in function and the primary key to show where various rows live on a given tablet.

Because there is not an existing primary key (PK) for `tbl_no_pk`, it is possible to add a one.

> IMPORTANT
> 
> Once you create a primary key on a table, you can not alter it.

Here are examples of how to define a primary key for a single column, using either consistent hash sharding or range sharding:
 - `PRIMARY KEY (k)` for hash  [DEFAULT]
 - `PRIMARY KEY (k hash)` for hash
 - `PRIMARY KEY (k asc)` for range

> NOTE
> 
> In this scenario, when you add a primary key to a table that does not have one, YugabyteDB will need to recreate the entire table behind the scenes and distribute the data according to PK sharding algorithm, range or hash. The previous table with the table id will show the table name as table_name_temp_old. The newly created table will have a new table id.

To rename the table and add a primary key using hash sharding, run the following cell:

In [ ]:
%%sql /* alter table rename AND add PK */

drop table if exists tbl_pk_hash;
-- rename table
alter table if exists tbl_no_pk rename to tbl_pk_hash;
-- add PK as hash
alter table tbl_pk_hash add primary key (k hash);

To view the definition of the renamed table, run the following cell:

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" #\d+
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

./bin/ysqlsh -d ${DB_NAME} -c "\d+ tbl_pk_hash"

#### View the Table details in the YB-Master web ui
In your web browser, open the following URL.

In [ ]:
#%% python, but prepared statements as sql magic
THIS_TABLE_NAME = 'tbl_pk_hash'
THIS_SCHEMA_NAME = 'public'
DB_NAME = MY_DB_NAME

## Comment out if local
view_gitpod_url = %sql select fn_get_table_id_url(:MY_YB_MASTER_HOST_GITPOD_URL,7000,:DB_NAME,:THIS_SCHEMA_NAME,:THIS_TABLE_NAME ) as view_gitpod_url
print (view_gitpod_url)

## Uncomment if local
# view_local_url = %sql select fn_get_table_id_url(:MY_HOST_IPv4_01,7000,:DB_NAME,:THIS_SCHEMA_NAME,:THIS_TABLE_NAME ) as view_local_url
# print (view_local_url)

##### Review the Column section
The column section shows details about each column in the table, index, or materialized view. In the column section for `tbl_pk_hash`, you will see:


| Column | ID	| Type |
|--------|------|------|
| k	         | 1	| int32 PARTITION KEY | 
| v	         | 2	| string NULLABLE NOT A PARTITION KEY | 

<br/>

Because there is a primary key defined for this table, there is no `ybrowid` column.

##### Partition Key
When YugabyteDB processes an insert query for a row of data into a table with a consistent hash sharding strategy, YugabyteDB uses the shard key (shown as `PARTITION KEY`) to distribute the data among the tablet leaders. 

With consistent hash sharding, a partitioning algorithm distributes data evenly and randomly across shards. By computing a consistent hash on the partition key (or keys) of a given row, YugabyteDB knows where to insert the row among the tablet leaders.



##### Review the Tablet section
The Tablet section shows the details for the existing tablets. Here is an abridged example of the Tablet section for the `tbl_pk_hash` table:

| Tablet ID |	Partition	| SplitDepth	| State	| Hidden	| Message	| RaftConfig|
|--|--|--|--|--|--|--|
| some_uuid_1 |	`hash_split: [0x5555, 0xAAAA)` |	0	| Running|	false| Tablet reported with an active leader	|<li>FOLLOWER: 127.0.0.1 <li>FOLLOWER: 127.0.0.3<li>LEADER: 127.0.0.2  |
| some_uuid_2	| `hash_split: [0xAAAA, 0xFFFF)`	| 0 |  Running |false |	Tablet reported with an active leader |<li>FOLLOWER: 127.0.0.1 <li>LEADER: 127.0.0.3 <li>FOLLOWER: 127.0.0.2 |
| some_uuid_3 <br>(tablet leader where the row lives) |	`hash_split: [0x0000, 0x5555)` |	0 |	Running | 	false	| Tablet reported with an active leader |	<li>LEADER: 127.0.0.1<li>FOLLOWER: 127.0.0.3<li>FOLLOWER: 127.0.0.2 |

##### What is the `hash_split` value in the Partition column? 

For a table with hash sharding, each tablet has a Partition value that shows a  `hash_split` . The `hash_split` details a range of hexadecimal values. 
```
hash_split: [0x0000, 0x5555)
hash_split: [0x5555, 0xAAAA)
hash_split: [0xAAAA, 0xFFFF)
```
The conversion of the above hexadecimal values to integer values is as follows:
```
0-21845
21845-43690
43690-65535
```

`tbl_pk_hash` uses consistent hash sharding with the partition key being `k`.  All tablets will store rows using the hash coded values for `k`.

#### q2a | yb_hash_code()
YSQL has an built-in function that shows the integer form of a hash coded value, `yb_hash_code()`. 

Using `yb_hash_code()`, you can view the hash code value of the primary key, `k`, for `tbl_pk_hash`. 

The resulting integer value places the row in the the hash_split range of the tablets. To help make the conversion from hexadecimal to integer easier to comprehend, run the following cell.

The query shows:
- k, the primary key value
- the `yb_hash_code()` integer for k
- and hash_split hexadecimal range in integer form

In [ ]:
%%sql /* query with hash buckets */
select '' as _
  , k
 -- , v
  , yb_hash_code(k::int) as k_hash_code
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( k::int),'hash_split: [0x0000, 0x5555)'::text) as col_0x0000_0x5555
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( k::int),'hash_split: [0x5555, 0xAAAA)'::text) as col_0x5555_0xAAAA
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( k::int),'hash_split: [0xAAAA, 0xFFFF)'::text) as col_0xAAAA_0xFFFF
from tbl_pk_hash
where 1=1 
-- and k=123
order by k asc
-- order by k desc
-- offset 330
-- offset 660
limit 30
;


##### q2a | Observations
The results show that the 30 returned rows are distributed among all of the hash_split ranges, meaning that each tablet leader is returning results. The results are sorted in an ascending order of K values.

##### q2a | Experiment
Question: 
- In the query cell above, comment out the `order by k asc` line. Rerun the query cell. What new observations can you make about the results?

Answer:
- The results are in the order of `k_hash_code`, not `k`.
- YugabyteDB only returns `k_hash_code` ordered results and these 30 row sare from the first hash_split range, `0-21845`.
- The `k_hash_code` ordering is how the internal LSM-Tree sorted the table inserts.
- For the given tablet, the LSM-tree persists to disk as Sorted Sequence Tables (SST files).
- The reads come from the SST file for the tablet. For this reason, you can rerun the query over and over again and still see the same "order" (`k_hash_code`) of "unordered" (`k`) results.


Question: 
- In the query above, with the `order by k` line already commented out, uncomment the `offset 330` line. Rerun the query cell. What observations can you make about the results?

Answer:
- YugabyteDB only returns `k_hash_code` ordered results from the beginning of the offset, which spans two hash_split ranges, indicating that the hash sharding is more or less evenly split among the tablets.


#### q2b | View the explain plan and metrics
Using the same query, you will generate an Explain Plan and the view the metrics.

Just like the section before, you will then modify and rerun the query to answer various questions.

In [ ]:
%%sql /* explain plan */
execute stmt_util_metrics_snap_reset;
explain (costs off, analyze, verbose) 
select '' as _
  , k
 -- , v
  , yb_hash_code(k::int) as k_hash_code
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( k::int),'hash_split: [0x0000, 0x5555)'::text) as col_0x0000_0x5555
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( k::int),'hash_split: [0x5555, 0xAAAA)'::text) as col_0x5555_0xAAAA
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( k::int),'hash_split: [0xAAAA, 0xFFFF)'::text) as col_0xAAAA_0xFFFF
from tbl_pk_hash
where 1=1 
-- and k=123
order by k asc
-- order by k desc
-- offset 330
-- offset 660
limit 30
;

##### q2b | Explain Plan (above ^^)
For the initial query with the `order by asc` clause, the Explain Plan shows that this is a `Seq Scan` query, and no index is being used:

`-> Seq Scan on public.tbl_pk_hash (actual time=2.626..7.200 rows=1000 loops=1)`

A `Seq Scan` results in potentially accessing all the rows in the table.  These means all tablet leaders will most likely process the query.

In [ ]:
%%sql
execute stmt_util_metrics_snap_table;

##### q2b | Metrics (above ^^)
In the initial query, the `Seq Scan` accesses all tablets leaders:

| row_name| 	rocksdb_number_db_seek | 	rocksdb_number_db_next | 
|--|--|--|
| db_ybu tbl_pk_hash link_table_id tablet_id_unq_1	 | 1 | 	689 |
| db_ybu tbl_pk_hash link_table_id tablet_id_unq_2	| 1	| 641 |
| db_ybu tbl_pk_hash link_table_id tablet_id_unq_3	| 1	| 667 |

##### q2b | Experiment
Question: 
- In the query cell above, comment out the `order by k` line. Rerun the query cell. What new observations can you make about the results?

Answer:
- Even though the Explain Plan shows that this is a `Seq Scan` query:
  - `-> Seq Scan on public.tbl_pk_hash (actual time=1.072..5.176 rows=30 loops=1`

The table scan is only for the tablet that contains the hash_split range where 30 rows live.
| row_name| 	rocksdb_number_db_seek | 	rocksdb_number_db_next | 
|--|--|--|
| db_ybu tbl_pk_hash link_table_id tablet_id_unq_1	 | 1 | 	62 |

- YugabyteDB returns the unordered results and from the first hash_split range, `0-21845`.
- The "unordered" results are actually returned as they were ordered in the LSM-Tree as Sorted Sequence Tables (SST files). For this reason, you can rerun the query over and over again and still see the same "order" of "unordered" results. 


Question: 
- With the `order by k` line commented out, what happens when you uncomment the `offset 330` lines and run the  Explain Plan and Metrics cells again? How does the output of Explain Plan compare to the Metrics output?


Answer:
- YugabyteDB returns results from the beginning of the offset, which, if the hash sharding is more or less evenly split, should span two hash_split ranges. Even though the Explain Plan shows that this is a `Seq Scan` query:
  - `-> Seq Scan on public.tbl_pk_hash (actual time=4.039..55.580 rows=360 loops=1)`

The table scan is for the tablets that contains the hash_split ranges. This means full table scans for two tablet leaders.

| row_name| 	rocksdb_number_db_seek | 	rocksdb_number_db_next | 
|--|--|--|
| db_ybu tbl_pk_hash link_table_id tablet_id_unq_1	 | 1 | 	689 |
| db_ybu tbl_pk_hash link_table_id tablet_id_unq_2	| 1	| 641 |


#### q2c | What's on disk?

Take a look at how YugabyteDB stores data in DocDB as tablets. But first, let's create the table and define a more complex primary key, one that include a partition key for hash sharding and a clustering key for ordering.


In [ ]:
%%sql /* alter table rename AND add PK */

drop table if exists tbl_pk_hash_ck_asc;

create table if not exists tbl_pk_hash_ck_asc (
  k int, 
  v1 text, 
  v2 text,
  v3 text,
  -- primary key (k hash)
  primary key (k hash, v1 asc)
  --primary key (k hash, v1 asc, v2 asc) 
);


insert into tbl_pk_hash_ck_asc (k,v1,v2,v3)
select 
 g.id,
 fn_random_chars(),
 fn_random_name(1,0),
 fn_random_name(0,1)
from generate_series(1, 100) AS g (id);

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" #\d+
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

./bin/ysqlsh -d ${DB_NAME} -c "\d+ tbl_pk_hash_ck_asc"

Save the table name as a variable.

In [ ]:
MY_OBJECT_NAME = "tbl_pk_hash_ck_asc"
%store MY_OBJECT_NAME


Get the `table_id` for the table.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_OBJECT_NAME"  --out MY_TABLE_ID

YB_PATH=${1}
DB_NAME=${2}
OBJECT_NAME=${3}

cd $YB_PATH
MY_TABLE_ID=$(./bin/ysqlsh -d ${DB_NAME} -t -c "select fn_get_table_id_from_pg('${DB_NAME}','public','${OBJECT_NAME}');")

echo $MY_TABLE_ID

Store the `table_id` for the table.

In [ ]:
%store MY_TABLE_ID
print(MY_TABLE_ID)

Get the `tablet_id` for the tablet leader for the selected node host.

In [ ]:
%%bash -s "$MY_DB_NAME" "$MY_OBJECT_NAME" "$MY_HOST_IPv4_01" --out MY_TABLET_ID

DB_NAME=${1}
OBJECT_NAME=$( echo "${2}" | tr -d " ")
HOST_IPv4=$( echo "${3}" | tr -d " ")
# can use WORKSPACE_URL
MY_URL="http://${HOST_IPv4}:8200/metrics"

TABLET_ID=`wget -cq  ${MY_URL}  -O - |  jq --raw-output ' .[] | select(.attributes.namespace_name=="'$DB_NAME'" and .type=="tablet" and .attributes.table_name=="'$OBJECT_NAME'") | {tablet_id: .id, metrics: .metrics[] | select(.name == ("is_raft_leader") ) | select(.value == 1) } | select(.tablet_id) | {tablet_id} | .tablet_id '`

echo ${TABLET_ID}

Store the `tablet_id` for the tablet leader.

In [ ]:
%store MY_TABLET_ID
print(MY_TABLET_ID)

Flush the WAL file to a SST file for the given table_id.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_HOST_IPv4_01" "$MY_TABLE_ID" 
YB_PATH=$( echo "${1}" | tr -d " ")
HOST_IPv4=$( echo "${2}" | tr -d " ")
TABLE_ID=$( echo "${3}" | tr -d " ")

cd $YB_PATH/bin

./yb-admin -init_master_addrs ${HOST_IPv4}:7100 flush_table_by_id ${TABLE_ID} 600

Dump and decode the SST file in human-readable form.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_YB_PATH_DATA" "$MY_TABLE_ID" "$MY_TABLET_ID" # Import file path of Yugabyte and DB name
YB_PATH=$( echo "${1}" | tr -d " ")
YB_PATH_DATA=$( echo "${2}" | tr -d " ")
TABLE_ID=$( echo "${3}" | tr -d " ")
TABLET_ID=$( echo "${4}" | tr -d " ")

cd $YB_PATH/bin/

TABLE_ID_PATH=${YB_PATH_DATA}/node-1/disk-1/yb-data/tserver/data/rocksdb/table-${TABLE_ID}/tablet-${TABLET_ID}

#ls -l  ${TABLE_ID_PATH}

./sst_dump --command=scan --file=${TABLE_ID_PATH} --output_format=decoded_regulardb 

You can go back to the beginning of  ***What's on disk?** an alter the Primary Key definition and run through the SST dump again. 

---
## Range Sharding
Range sharding splits the rows of a table into contiguous ranges that respect the sort order of the table based on the primary key column values. Typically, a range sharded table begins with only one tablet. With more row insertions, the single tablet eventually splits into two or more tablets. As the data volume for the range sharded table continues to grow, so does the number of tablets for the table. YugabyteDB balances the number of tablet leaders across the YB-TServers in the cluster. 
Here is an illustrations of range sharding:

<img  style="background-color:#FFF" src="assets/range_sharding.png" >

A query that specifies a range of values for the primary key (shard / partition key) in the query predicate, will often benefit from the source table using a range sharding strategy.

### q3 | Create a table with range sharding
By running the following cell, you will create a table with a primary key that employs range sharding:

In [ ]:
%%sql  /* create table no PK and insert rows */
drop table if exists tbl_pk_range;

create table if not exists tbl_pk_range (k int, v text, primary key (k asc)) ;

insert into tbl_pk_range (k,v)
select g.id, format('%s%s',chr(97+CAST(random() * 25 AS INTEGER)),chr(97+CAST(random() * 25 AS INTEGER)))
from generate_series(1, 1000) AS g (id);

#### View in the YB-Master web ui for the table details
To view the table details, copy and paste the following URL in your web browser:

In [ ]:
#%% python, but prepared statements as sql magic
THIS_TABLE_NAME = 'tbl_pk_range'
THIS_SCHEMA_NAME = 'public'
DB_NAME = MY_DB_NAME

## Comment out if local
view_gitpod_url = %sql select fn_get_table_id_url(:MY_YB_MASTER_HOST_GITPOD_URL,7000,:DB_NAME,:THIS_SCHEMA_NAME,:THIS_TABLE_NAME ) as view_gitpod_url
print (view_gitpod_url)

## Uncomment if local
# view_local_url = %sql select fn_get_table_id_url(:MY_HOST_IPv4_01,7000,:DB_NAME,:THIS_SCHEMA_NAME,:THIS_TABLE_NAME ) as view_local_url
# print (view_local_url)

##### Review the Column section
In the column section for `tbl_pk_range`, you will see:


| Column | ID	| Type |
|--------|------|------|
| k	         | 1	| int32 NOT NULL NOT A PARTITION KEY | 
| v	         | 2	| string NULLABLE NOT A PARTITION KEY | 

<br/>

> Note
>  
> There is no PARTITION KEY (shard key) for the `tbl_pk_range` table, which indicates the use of another sharding strategy. `k`, the primary key, only shows that the column can not contain a NULL value`.


##### Review the Tablet section
The Tablet section shows the details for the existing tablets. Here is an abridged example of the Tablet section for the `tbl_pk_hash` table:

| Tablet ID |	Partition	| SplitDepth	| State	| Hidden	| Message	| RaftConfig|
|--|--|--|--|--|--|--|
| some_uuid_1<br>`42ca468817c145c29145752718b7c6a7` |	`range: [<start>, <end>)` |	0	| Running|	false| Tablet reported with an active leader	|<li>FOLLOWER: 127.0.0.1 <li>FOLLOWER: 127.0.0.3<li>LEADER: 127.0.0.2  |

YugabyteDB will automatically split this tablet based on the size of the table on disk. The following global flags determine this behavior:

```
--tablet_force_split_threshold_bytes=107374182400 --> 10240 MB
--tablet_split_high_phase_shard_count_per_node=24
--tablet_split_high_phase_size_threshold_bytes=10737418240 --> 10240 MB
--tablet_split_low_phase_shard_count_per_node=8
--tablet_split_low_phase_size_threshold_bytes=536870912 --> 512 MB
--tablet_split_size_threshold_bytes=0
```

The low phase indicate the threshold for the initial splits of the tablet. With more data volume, the threshold increases from 512 MB to over 10 GBs.

#### q3a | Natural order
Unlike the consistent hash sharding of `tbl_pk_hash`, `tbl_pk_range` preserves the order of the PK. To view the difference, run the following query cell:

In [ ]:
%%sql /* query */
select '' as _
  , k
from tbl_pk_range
where 1=1 
-- and k = 123
and k between 100 and 200
-- order by k desc
limit 30
;

The above result reveals that range sharding stores data in the specified order of the primary key definition, which by default is `asc`, ascending order.

To view the explain plan for the query, run the following cell:

In [ ]:
%%sql /* explain plan */
execute stmt_util_metrics_snap_reset;
explain (costs off, analyze, verbose) 
select '' as _
  , k
from tbl_pk_range
where 1=1 
-- and k = 123
and k between 100 and 200
-- order by k desc
limit 30
;

##### q3a | Explain Plan (above ^^)

The Explain Plan shows that this query uses the PK index and reads just one row..
- `-> Index Scan using tbl_pk_range_pkey on public.tbl_pk_range (actual time=1.972..1.983 rows=30 loops=1)`

The Index Condition reflects the query predicate.

- `Index Cond: ((tbl_pk_range.k >= 100) AND (tbl_pk_range.k <= 200))`

To view the tablet metrics, run the following cell wit the prepared statement:

In [ ]:
%%sql
execute stmt_util_metrics_snap_table;

##### q3a | Metrics (above ^^)

The Explain Plan revealed that the query uses the PK index. Because there is only one tablet at this time for the table with range sharding, the Metrics report shows that the query accesses a single tablet. There is just one seek for the offset and then about two reads per row (limit 30).

##### q3a | Experiment

Question: 
- Uncomment the `order by k desc` line. Rerun the Explain Plan and the Metrics prepared statement. What new observations can you make? 


Answer:
- The Explain Plan shows that query uses `-> Index Scan Backward` for the PK index.
- The Metrics shows that the Index Scan Backward operations resulted in 64 seeks and only 62 reads. For thirty rows (`limit 30`), this means that there a two seeks per each backwards operation!


| row_name| 	rocksdb_number_db_seek | 	rocksdb_number_db_next | 
|--|--|--|
| db_ybu tbl_pk_range link_table_id tablet_id_unq_1	 | 64 | 	62 |


When fetching rows in reverse order of the index, with `Index Scan Backward`, there is no next call to go backwards. Instead, there are two seeks to go backwards. For this reason, the range sharding order of the primary key (`asc` or `desc`) may not be optimal for queries with an `Index Scan Backward` plan. 


---
## Comparing hash and range sharding for tables
For this challenge, you will the Explain Plan and Metrics report for two sets of queries, each with tables that implement hash and range sharding.

### q4a | Query with single-key predicate
To begin, run the following cell:

In [ ]:
%%sql /* explain plan */
execute stmt_util_metrics_snap_reset;
explain (costs off, analyze, verbose) 
select '' as _
  , k
from tbl_pk_hash
-- from tbl_pk_range
where 1=1 
and k = 123
-- and k between 100 and 200
-- order by k desc
-- limit 30
;


#### q4a | Explain Plan (above ^^)

The Explain Plan shows that this query uses the PK index and reads just one row..
- `Index Scan using tbl_pk_hash_pkey on public.tbl_pk_hash (actual time=1.428..1.429 rows=1 loops=1)`

The Index Condition reflects the query predicate.

- `Index Cond: (tbl_pk_hash.k = 123)`

To view the tablet metrics, run the following cell wit the prepared statement:

In [ ]:
%%sql
execute stmt_util_metrics_snap_table;

#### q4a | Metrics (above ^^)

The Explain Plan revealed that the query uses the PK index. 
The Metrics report shows that only one tablet is accessed, with one seek of the offset and two reads.

#### q4a | Experiment

Question: 
- In the Explain Plan cell, first comment out `from tbl_pk_hash` and then uncomment `from tbl_pk_range`. Rerun the Explain Plan and the Metrics prepared statement. What new observations can you make? 


Answer:
- Besides being different tables and tablets, the Explain Plan and the Metrics report are essentially identical. Both Explain Plans use the PK index and both access one tablet.



### q4b | Query with key range in predicate
To begin, run the following cell:

In [ ]:
%%sql /* explain plan */
execute stmt_util_metrics_snap_reset;
explain (costs off, analyze, verbose) 
select '' as _
  , k
from tbl_pk_hash
-- from tbl_pk_range
where 1=1 
-- and k = 123
and k between 100 and 200
order by k asc
limit 30
;


#### q4b | Explain Plan (above ^^)

The initial Explain Plan shows that this query does not use the PK index for the range value query.
- `-> Seq Scan on public.tbl_pk_hash (actual time=2.910..2.956 rows=101 loops=1`

The `Filter` indicates that the YB-TServer for the client connection filtered the results, removing 899 rows. 

- `Filter: ((tbl_pk_hash.k >= 100) AND (tbl_pk_hash.k <= 200))`
- `Rows Removed by Filter:  899`

This means that the tablets (RocksDB) sent all reads to the YB-TServer to be filtered.

To view the tablet metrics, run the following cell wit the prepared statement:

In [ ]:
%%sql
execute stmt_util_metrics_snap_table;

#### q4b | Metrics (above ^^)

The initial Explain Plan reveals that the query requires a `Seq Scan`, which is full table scan. 
The Metrics report shows that query accesses all three tablets, with an 1 seek per tablet and over 600 reads.

#### q4b | Experiment

Question: 
- In the Explain Plan cell, first comment out `from tbl_pk_hash` and then uncomment `from tbl_pk_range`. Rerun the Explain Plan and the Metrics prepared statement. What new observations can you make? 


Answer:
- The `from tbl_pk_range` is an extremely efficient query.
- The query uses the PK index, `Index Scan using tbl_pk_range_pkey on public.tbl_pk_range (actual time=0.816..0.825 rows=30 loops=1`.
- The YB-TServer for the client connection does not filter any rows. 
- Because the table employs range sharding, the query access the single tablet for the table using 1 seek for the offset, and about two reads per row (`limit 30`).



---
## Summary
In this notebook, you learned about how YugabyteDB uses two distinct sharding strategies for storing data in a distributed manner for tables in tablets:
- consistent hash sharding
- range sharding

You specify the sharding strategy in the Primary Key definition of a table: `hash` for hash sharding, and either `asc` or `desc` for range sharding.
By default, YSQL for YugabyteDB employs consistent hash sharding, even if a primary key is not defined.

### Consistent hash sharding
With consistent hash sharding, YugabyteDB evenly and randomly distributes data across tablets using a partitioning algorithm. By computing a consistent hash on the partition column values of that row, YugabyteDB places each row of the table into a tablet. By using the `yb_hash_code()` built-in function, you can view the integer form of a hash coded value.

#### Benefits of hash sharding
Because consistent hash sharding distributes data evenly across all the nodes in the cluster while also retaining the ability to horizontally scale (add or remove nodes) in a YugabyteDB cluster, the strategy is ideal for massively scalable workloads that require shard key predicate queries.

#### Concerns with hash sharding
However, performing range queries could be inefficient. Examples of range queries are finding rows greater than a lower bound or less than an upper bound (as opposed to point lookups).


### Range sharding
Range sharding involves splitting the rows of a table into contiguous ranges that respect the sort order of the table based on the primary key column values. YugabyteDB initially creates a single tablet leader for a table. with range sharding. As the data volume increases for the table, YugabyteDB dynamically splits the initial tablet into two or moore tablets. 

#### Benefits of range sharding
Range harding allows efficiently querying a range of rows by the primary key values. Examples of such a query is to look up all keys that lie between a lower bound and an upper bound.


#### Concerns with range sharding
Range sharding can introduce a number of issues in practice and at scale. Starting out with a single tablet leader implies only a single node is handling all the user queries, making this node in the cluster "hot" in terms of CPU, Memory,  Network, and Disk utilization. To alleviate this side effect, the data volume must grow significantly in order for the tablet to split. Another concern is that the global ordering of keys across all the tablets itself generates hot spots. As a result, certain tablet will naturally have more activity than others, resulting in an node with increased utilization of CPU, Memory, Network, and Disk.

---
# 🌟🌟🌟 Well done! 
In this notebook, you completed the following:
- Hash Sharding
- Range Sharding
- Comparing hash and range sharding
- Summary

## 😊 Next Steps!
Join our learning community at Yugabyte University and achieve recognition for your achievements. Enrollment and certification is FREE. Sign up today at:
- [https://university.yugabyte.com/users/sign_up](https://university.yugabyte.com/users/sign_up)
